In [126]:
#importing the dataset
import pandas as pd

hdr = list(range(0, 57))
temp_df = pd.read_table('ML Engineer Candidate Prompt Dataset.txt', delimiter='\t',names = hdr,  low_memory = False)
print(temp_df.head(5))

                      0       1       2       3       4       5       6   \
initial_weight        60      70     210     540     900     660     390   
age_at_diagnosis   25041   21901   30643   22575   32415   26971   20318   
vital_status       Alive   Alive    Dead   Alive    Dead   Alive   Alive   
age_at_index          68      59      83      61      88      73      55   
days_to_birth     -25041  -21901  -30643  -22575  -32415  -26971  -20318   

                      7       8       9   ...      47      48      49      50  \
initial_weight       350    1010     290  ...     400     170     300     340   
age_at_diagnosis   22650   27688   25791  ...   29658   23826   25386   26054   
vital_status        Dead    Dead    Dead  ...    Dead    Dead   Alive   Alive   
age_at_index          62      75      70  ...      81      65      69      71   
days_to_birth     -22650  -27688  -25791  ...  -29658  -23826  -25386  -26054   

                      51      52      53      54      55

In [127]:
#transpose table
temp_df = temp_df.transpose()
print(temp_df.dtypes)

initial_weight       object
age_at_diagnosis     object
vital_status         object
age_at_index         object
days_to_birth        object
                      ...  
ENSG00000288669.1    object
ENSG00000288670.1    object
ENSG00000288671.1    object
ENSG00000288674.1    object
ENSG00000288675.1    object
Length: 60670, dtype: object


In [128]:
#saving fixed table
temp_df.to_csv('fixed_table')
print("File successfully saved")

File successfully saved


In [129]:
#opening the fixed table
df = pd.read_csv('fixed_table', low_memory=False)
df.drop(columns=df.columns[0], axis=1,inplace=True)
print(df.dtypes)

initial_weight        int64
age_at_diagnosis      int64
vital_status         object
age_at_index          int64
days_to_birth         int64
                      ...  
ENSG00000288669.1     int64
ENSG00000288670.1     int64
ENSG00000288671.1     int64
ENSG00000288674.1     int64
ENSG00000288675.1     int64
Length: 60670, dtype: object


In [130]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

df['demographic_id'] = le.fit_transform(df['demographic_id'])
df['vital_status'] = le.fit_transform(df['vital_status'])
df['paper_clinical_stage'] = le.fit_transform(df['paper_clinical_stage'])
df['year_of_death'] = le.fit_transform(df['year_of_death'])
df['days_to_death'] = le.fit_transform(df['days_to_death'])
#E.g. vital_status has been encoded to be 0 for alive and 1 for dead
print(df['vital_status'].unique())

[0 1]


In [131]:
#preparing the data in X and y sets
X = df.drop(columns = ['initial_weight','vital_status', 'year_of_death','days_to_death'])
y = df['vital_status']
print(X.head(3))
print(y.head(3))

   age_at_diagnosis  age_at_index  days_to_birth  year_of_birth  \
0             25041            68         -25041           1940   
1             21901            59         -21901           1952   
2             30643            83         -30643           1926   

   demographic_id  paper_clinical_stage  ENSG00000000003.15  \
0              45                     0                4511   
1              16                     0                6035   
2              11                     2                5583   

   ENSG00000000005.6  ENSG00000000419.13  ENSG00000000457.14  ...  \
0                402                1433                 201  ...   
1                 23                1886                 775  ...   
2                  8                2542                 946  ...   

   ENSG00000288661.1  ENSG00000288662.1  ENSG00000288663.1  ENSG00000288665.1  \
0                  0                  0                  4                  0   
1                  0                  0

In [168]:
#dividing data into sets to be trained and tested at a 20% test size
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size = 0.2)

In [169]:
#Scale the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
print(X_train)

[[ 1.04942169  1.08117346 -1.04942169 ...  0.          1.21857244
   0.21663464]
 [-0.97772819 -0.9251278   0.97772819 ...  0.         -1.52321555
   0.73152331]
 [-0.12664107 -0.1226073   0.12664107 ...  0.          1.44705477
   0.01067917]
 ...
 [-0.54775188 -0.52386755  0.54775188 ...  0.         -0.15232156
   1.79562657]
 [ 1.81323912  1.7833789  -1.81323912 ...  0.         -0.60928622
  -0.53853541]
 [ 1.32231257  1.28180358 -1.32231257 ...  0.         -0.83776855
  -0.91612044]]


In [170]:
#Preparing and training the models
#Creating a function to train all models at once

def models(X_train, y_train):
    #0 = Decision Tree Classifier
    from sklearn.tree import DecisionTreeClassifier
    dtc = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    dtc.fit(X_train,y_train)
    
    #1 = Random Forest Classifier
    from sklearn.ensemble import RandomForestClassifier
    rfc = RandomForestClassifier(n_estimators=10, criterion = 'entropy', random_state = 0)
    rfc.fit(X_train,y_train)

    #2 = SVC (linear k)
    from sklearn.svm import SVC
    svc_ln = SVC(kernel='linear', random_state = 0)
    svc_ln.fit(X_train,y_train)
    
    #3 = SVC (RBF)
    from sklearn.svm import SVC
    svc_rbf = SVC(kernel='rbf', random_state = 0)
    svc_rbf.fit(X_train,y_train)

    #4 = Logistic Regression
    from sklearn.linear_model import LogisticRegression
    lreg = LogisticRegression(random_state = 0)
    lreg.fit(X_train,y_train)

    #5 = Gaussian NB
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train,y_train)

    #6 = K-Neighbours
    from sklearn.neighbors import KNeighborsClassifier
    knc = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
    knc.fit(X_train,y_train)
    
    #print the accuracy of each model
    print('0 - Decision Tree Classifier', dtc.score(X_train, y_train))
    print('1 - Random Forest Classifier', rfc.score(X_train, y_train))
    print('2 - SVC (linear k)', svc_ln.score(X_train, y_train))
    print('3 - SVC (RBF)', svc_rbf.score(X_train, y_train))
    print('4 - Logistic Regression', lreg.score(X_train, y_train))
    print('5 - Gaussian NB', gnb.score(X_train, y_train))
    print('6 - K-Neighbours', knc.score(X_train, y_train))
    
    return dtc, rfc, svc_ln, svc_rbf, lreg, gnb, knc

model = models(X_train, y_train)


0 - Decision Tree Classifier 1.0
1 - Random Forest Classifier 1.0
2 - SVC (linear k) 1.0
3 - SVC (RBF) 0.9555555555555556
4 - Logistic Regression 1.0
5 - Gaussian NB 1.0
6 - K-Neighbours 0.8


In [171]:
#Saving models
import joblib
joblib.dump(model[0], 'Decision Tree Classifier')
joblib.dump(model[1], 'Random Forest Classifier')
joblib.dump(model[2], 'SVC (linear k)')
joblib.dump(model[3], 'SVC (RBF)')
joblib.dump(model[4], 'Logistic Regression')
joblib.dump(model[5], 'Gaussian NB')
joblib.dump(model[6], 'K-Neighbours')
print("Models saved successfully")

Models saved successfully


In [172]:
#Loading models
import joblib
import numpy as np
models = np.empty(7, dtype=object)
models[0] = joblib.load('Decision Tree Classifier')
models[1] = joblib.load('Random Forest Classifier')
models[2] = joblib.load('SVC (linear k)')
models[3] = joblib.load('SVC (RBF)')
models[4] = joblib.load('Logistic Regression')
models[5] = joblib.load('Gaussian NB')
models[6] = joblib.load('K-Neighbours')

model = models
print("Models Loaded Successfully")

Models Loaded Successfully


In [173]:
from sklearn.metrics import confusion_matrix

for i in range(len(model)):
    cm = confusion_matrix(y_test, model[i].predict(X_test))
    
    TN, FP, FN, TP = confusion_matrix(y_test, model[i].predict(X_test)).ravel()
    
    score = (TP+TN)/(TP+TN+FN+FP)
    print(cm)
    print('Model[{}] Testing Accuracy = "{}"'.format(i, score),"\n")

[[3 4]
 [2 3]]
Model[0] Testing Accuracy = "0.5" 

[[5 2]
 [3 2]]
Model[1] Testing Accuracy = "0.5833333333333334" 

[[0 7]
 [1 4]]
Model[2] Testing Accuracy = "0.3333333333333333" 

[[0 7]
 [0 5]]
Model[3] Testing Accuracy = "0.4166666666666667" 

[[0 7]
 [1 4]]
Model[4] Testing Accuracy = "0.3333333333333333" 

[[0 7]
 [0 5]]
Model[5] Testing Accuracy = "0.4166666666666667" 

[[0 7]
 [1 4]]
Model[6] Testing Accuracy = "0.3333333333333333" 

